In [1]:
#pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [198]:
import pandas as pd
import numpy as np
#排程
import schedule
import time
from pandas import Timestamp

In [78]:
solar_new = pd.read_csv(f'dataset/solar_汙水廠(test_new).csv', header=None)
history_15m = pd.read_csv(f'./dataset/solar_汙水廠_newbig_sort(history_15m).csv', header=None)
dfXlen = len(solar_new)
solar_new = solar_new.iloc[0:dfXlen,:]
solar_new = solar_new.drop(columns=[5, 9, 13, 14, 20, 21, 22])#刪除IBM欄位

C:\Users\user\AppData\Local\Temp\ipykernel_22752\1714564243.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  history_15m = pd.read_csv(f'./dataset/solar_汙水廠_newbig_sort(history_15m).csv', header=None)


In [266]:
solar_new_noheader = solar_new[0]
solar_new_noheader
solar_new_noheader[1:] = pd.to_datetime(solar_new_noheader[1:], format='%Y/%m/%d %H:%M')
solar_new_noheader = solar_new_noheader[1:]
solar_new_noheader
# 取代原本的值
solar_new_noheader.loc[0] ='TIME_TO_INTERVAL'
solar_new[0] = solar_new_noheader

#取solar_汙水廠(test_new)的第一筆TIME_TO_INTERVAL
new_first_col = solar_new[0]
new_first_row = new_first_col[1]
#因new_first_row是Timestamp,需轉時間字串
new_first_row = new_first_row.strftime('%Y-%m-%d %H:%M:%S')

#取solar_汙水廠_newbig_sort(history_15m)的最後一筆TIME_TO_INTERVAL
history_row = history_15m.iloc[-1]
history_col = history_row[0]

#創建空的dataframe
solar_new_dataframe = [
    {'TIME_TO_INTERVAL'},
    {'Power'},
    {'Radiation'},
    {'ClearSkyRadiation'},
    {'Radiation(SDv3)(CWB)'},
    {'Radiation(SDv3)(OWM)'},
    {'Radiation(MSM)'},
    {'Radiation(today)(CWB)'},
    {'Radiation(today)(OWM)'},
    {'WeatherType(CWB)'},
    {'WeatherType(pred)(CWB)'},
    {'WeatherType(OWM)'},
    {'WeatherType(pred)(OWM)'},
    {'ApparentTemperature(pred)[CWB]'},
    {'Temperature(pred)[CWB]'},
    {'RelativeHumidity(pred)[CWB]'},
    {'FeelsLikeTemperature(pred)[OWM]'},
    {'Temperature(pred)[OWM]'},
    {'RelativeHumidity(pred)[OWM]'}
]

# 將 solar_new_dataframe 中的元素轉換為列表
cols = [list(x)[0] for x in solar_new_dataframe]

# 創建空的 dataframe
solar_new_dataframe = pd.DataFrame(columns=cols)

#假設新的一小時大表第一筆資料時間大於15分鐘表的最後一筆資料
if new_first_row > history_col:
    #整個時間欄
    time_all = solar_new.iloc[1:,0]  
    for j in range(0,dfXlen-1):
        #每隔15分鐘產生一個時間，共產生4個
        time_all_new = time_all.iloc[j]
        time_str = time_all_new.strftime('%Y-%m-%d %H:%M:%S')
        df_datatime = pd.date_range(start=time_str, periods=4, freq='15min')
        #第j+1行(一小時的),j欄是標題欄
        solar_new_original =solar_new.loc[j+1,:]
        #第一筆位置為0~3,第一筆後要從位置3之後開始存資料
        if j == 0:
            # 添加第一行數據
            solar_new_dataframe.loc[j, 'TIME_TO_INTERVAL'] = df_datatime[0]
            solar_new_dataframe.loc[j, 'Power'] = solar_new_original[1]
            solar_new_dataframe.loc[j, 'Radiation'] = solar_new_original[2]
            solar_new_dataframe.loc[j, 'ClearSkyRadiation'] = solar_new_original[3]
            solar_new_dataframe.loc[j, 'Radiation(SDv3)(CWB)'] = solar_new_original[4]
            solar_new_dataframe.loc[j, 'Radiation(SDv3)(OWM)'] = solar_new_original[6]
            solar_new_dataframe.loc[j, 'Radiation(MSM)'] = solar_new_original[7]
            solar_new_dataframe.loc[j, 'Radiation(today)(CWB'] = solar_new_original[8]
            solar_new_dataframe.loc[j, 'Radiation(today)(OWM'] = solar_new_original[10]
            solar_new_dataframe.loc[j, 'WeatherType(CWB)'] = solar_new_original[11]
            solar_new_dataframe.loc[j, 'WeatherType(pred)(CWB)'] =solar_new_original[12]
            solar_new_dataframe.loc[j, 'WeatherType(OWM)'] = solar_new_original[15]
            solar_new_dataframe.loc[j, 'WeatherType(pred)(OWM)'] = solar_new_original[16]
            solar_new_dataframe.loc[j, 'ApparentTemperature(pred)[CWB]'] =solar_new_original[17]
            solar_new_dataframe.loc[j, 'Temperature(pred)[CWB]'] = solar_new_original[18]
            solar_new_dataframe.loc[j, 'RelativeHumidity(pred)[CWB]'] = solar_new_original[19]
            solar_new_dataframe.loc[j, 'FeelsLikeTemperature(pred)[OWM]'] =solar_new_original[23]
            solar_new_dataframe.loc[j, 'Temperature(pred)[OWM]'] = solar_new_original[24]
            solar_new_dataframe.loc[j, 'RelativeHumidity(pred)[OWM]'] =solar_new_original[25]
            # 添加第2到4行數據(00:15~00:45)
            solar_new_dataframe.loc[j+1, 'TIME_TO_INTERVAL'] = df_datatime[1]
            solar_new_dataframe.loc[j+2, 'TIME_TO_INTERVAL'] = df_datatime[2]
            solar_new_dataframe.loc[j+3, 'TIME_TO_INTERVAL'] = df_datatime[3]
        else:
            # 添加第4行後數據
            solar_new_dataframe.loc[j+(j*3), 'TIME_TO_INTERVAL'] = df_datatime[0]
            solar_new_dataframe.loc[j+(j*3), 'Power'] = solar_new_original[1]
            solar_new_dataframe.loc[j+(j*3), 'Radiation'] = solar_new_original[2]
            solar_new_dataframe.loc[j+(j*3), 'ClearSkyRadiation'] = solar_new_original[3]
            solar_new_dataframe.loc[j+(j*3), 'Radiation(SDv3)(CWB)'] = solar_new_original[4]
            solar_new_dataframe.loc[j+(j*3), 'Radiation(SDv3)(OWM)'] = solar_new_original[6]
            solar_new_dataframe.loc[j+(j*3), 'Radiation(MSM)'] = solar_new_original[7]
            solar_new_dataframe.loc[j+(j*3), 'Radiation(today)(CWB'] = solar_new_original[8]
            solar_new_dataframe.loc[j+(j*3), 'Radiation(today)(OWM'] = solar_new_original[10]
            solar_new_dataframe.loc[j+(j*3), 'WeatherType(CWB)'] = solar_new_original[11]
            solar_new_dataframe.loc[j+(j*3), 'WeatherType(pred)(CWB)'] =solar_new_original[12]
            solar_new_dataframe.loc[j+(j*3), 'WeatherType(OWM)'] = solar_new_original[15]
            solar_new_dataframe.loc[j+(j*3), 'WeatherType(pred)(OWM)'] = solar_new_original[16]
            solar_new_dataframe.loc[j+(j*3), 'ApparentTemperature(pred)[CWB]'] =solar_new_original[17]
            solar_new_dataframe.loc[j+(j*3), 'Temperature(pred)[CWB]'] = solar_new_original[18]
            solar_new_dataframe.loc[j+(j*3), 'RelativeHumidity(pred)[CWB]'] = solar_new_original[19]
            solar_new_dataframe.loc[j+(j*3), 'FeelsLikeTemperature(pred)[OWM]'] =solar_new_original[23]
            solar_new_dataframe.loc[j+(j*3), 'Temperature(pred)[OWM]'] = solar_new_original[24]
            solar_new_dataframe.loc[j+(j*3), 'RelativeHumidity(pred)[OWM]'] =solar_new_original[25]
            # 添加第4行的3行數據
            solar_new_dataframe.loc[j+(j*3)+1, 'TIME_TO_INTERVAL'] = df_datatime[1]
            solar_new_dataframe.loc[j+(j*3)+2, 'TIME_TO_INTERVAL'] = df_datatime[2]
            solar_new_dataframe.loc[j+(j*3)+3, 'TIME_TO_INTERVAL'] = df_datatime[3] 
    solar_new_dataframe.to_csv(f'dataset/solar_汙水廠_newbig(solar(new)_history).csv', index=None)
    
else:
    print('NO new data')


0         TIME_TO_INTERVAL
1      2023-01-26 00:00:00
2      2023-01-26 01:00:00
3      2023-01-26 02:00:00
4      2023-01-26 03:00:00
              ...         
143    2023-01-31 22:00:00
144    2023-01-31 23:00:00
145    2023-02-01 00:00:00
146    2023-02-01 01:00:00
147    2023-02-01 02:00:00
Name: 0, Length: 148, dtype: object


In [ ]:
import schedule
import time
def job():
    print("工作1")
def job2():
    print('工作2')
def job3():
    print('工作3')
# 每(n)秒/分/時/天/週執行(job)
schedule.every(10).seconds.do(job)
schedule.every(1).minutes.do(job)
schedule.every(3).hours.do(job)
schedule.every(3).days.do(job)
schedule.every(3).weeks.do(job)
# at:每分鐘的第(n)秒時執行
schedule.every().minute.at(":23").do(job3)
while True:
    schedule.run_pending()